# Market Basket Analysis using Association Rules

mlxtend library includes apriori algorithm

In [4]:
!pip install mlxtend

In [5]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

df = pd.read_excel('online_retail_II.xlsx')
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


Exploring the data

In [7]:
df.columns

Index(['Invoice', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'Price', 'Customer ID', 'Country'],
      dtype='object')

In [13]:
df.Country.unique()
len(df.Country.unique())

40

In [12]:
len(df.StockCode.unique())

4632

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525461 entries, 0 to 525460
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      525461 non-null  object        
 1   StockCode    525461 non-null  object        
 2   Description  522533 non-null  object        
 3   Quantity     525461 non-null  int64         
 4   InvoiceDate  525461 non-null  datetime64[ns]
 5   Price        525461 non-null  float64       
 6   Customer ID  417534 non-null  float64       
 7   Country      525461 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 32.1+ MB


In [18]:
df.describe()

,Quantity,Price,Customer ID
count,525461.000000,525461.000000,417534.000000
mean,10.337667,4.688834,15360.645478
std,107.424110,146.126914,1680.811316
min,-9600.000000,-53594.360000,12346.000000
25%,1.000000,1.250000,13983.000000
50%,3.000000,2.100000,15311.000000
75%,10.000000,4.210000,16799.000000
max,19152.000000,25111.090000,18287.000000


Cleaning the data

In [24]:
index_names = df[(df.StockCode == "ADJUST") | (df.StockCode == "ADJUST2") | (df.StockCode == "BANK CHARGES") | (df.StockCode == "C2") | (df.StockCode == "D") |  (df.StockCode == "M") | (df.StockCode == "POST") | (df.StockCode == "TEST001") | (df.StockCode == "TEST002")].index
df.drop(index_names, inplace = True)

In [28]:
df.describe()

,Quantity,Price,Customer ID
count,511603.000000,511603.000000,406339.000000
mean,11.735856,3.402886,15373.632602
std,93.097998,113.661652,1677.371303
min,1.000000,-53594.360000,12346.000000
25%,1.000000,1.250000,14004.000000
50%,3.000000,2.100000,15326.000000
75%,11.000000,4.210000,16814.000000
max,19152.000000,13541.330000,18287.000000


In [30]:
index_names2 = df[ df.Quantity <1 ].index
df.drop(index_names2, inplace = True)

In [31]:
df.describe()

,Quantity,Price,Customer ID
count,511603.000000,511603.000000,406339.000000
mean,11.735856,3.402886,15373.632602
std,93.097998,113.661652,1677.371303
min,1.000000,-53594.360000,12346.000000
25%,1.000000,1.250000,14004.000000
50%,3.000000,2.100000,15326.000000
75%,11.000000,4.210000,16814.000000
max,19152.000000,13541.330000,18287.000000


In [37]:
df.dropna(axis = 0, subset =['Invoice'], inplace = True)
df['Invoice'] = df['Invoice'].astype('str')

In [38]:
df = df[~df['Invoice'].str.contains('C')]

In [36]:
df.describe()

,Quantity,Price,Customer ID
count,511603.000000,511603.000000,406339.000000
mean,11.735856,3.402886,15373.632602
std,93.097998,113.661652,1677.371303
min,1.000000,-53594.360000,12346.000000
25%,1.000000,1.250000,14004.000000
50%,3.000000,2.100000,15326.000000
75%,11.000000,4.210000,16814.000000
max,19152.000000,13541.330000,18287.000000


## Apriori

In [40]:
basket_France = (df[df['Country'] =="France"]
          .groupby(['Invoice', 'StockCode'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('Invoice'))

In [41]:
basket_UK = (df[df['Country'] =="United Kingdom"]
          .groupby(['Invoice', 'StockCode'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('Invoice'))

hotcode encoding 

In [42]:
def hot_encode(x):
    if(x<= 0):
        return 0
    if(x>= 1):
        return 1

In [43]:
basket_encoded = basket_France.applymap(hot_encode)
basket_France = basket_encoded

In [44]:
basket_France.head()

StockCode,10002,10120,16012,16014,16016,16043,16046,16047,16048,16053,...,90059B,90059C,90059D,90059E,90059F,90209A,90214C,90214E,90214L,90214S
Invoice,,,,,,,,,,,,,,,,,,,,,
489439,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
489557,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
489883,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
490139,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
490152,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [45]:
basket_encoded = basket_UK.applymap(hot_encode)
basket_UK = basket_encoded
basket_UK.head()

StockCode,10002,10080,10109,10120,10125,10133,10134,10135,10138,11001,...,gift_0001_10,gift_0001_20,gift_0001_30,gift_0001_40,gift_0001_50,gift_0001_60,gift_0001_70,gift_0001_80,gift_0001_90,m
Invoice,,,,,,,,,,,,,,,,,,,,,
489434,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
489435,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
489436,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
489437,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
489438,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


building the model

In [50]:
frequent_items = apriori(basket_France, min_support = 0.05, use_colnames = True)
print(frequent_items.head(20))

     support itemsets
0   0.110132  (10002)
1   0.052863  (20676)
2   0.066079  (20679)
3   0.083700  (20682)
4   0.083700  (20685)
5   0.096916  (20712)
6   0.140969  (20719)
7   0.202643  (20724)
8   0.176211  (20725)
9   0.132159  (20726)
10  0.123348  (20749)
11  0.251101  (20750)
12  0.057269  (21035)
13  0.070485  (21039)
14  0.132159  (21080)
15  0.140969  (21086)
16  0.127753  (21094)
17  0.127753  (21121)
18  0.070485  (21122)
19  0.052863  (21124)


In [49]:
rules = association_rules(frequent_items, metric ="lift", min_threshold = 1)
rules = rules.sort_values(['confidence', 'lift'], ascending = [False, False])
print(rules.head(10))

        antecedents consequents  antecedent support  consequent support  \
471  (22555, 22551)     (22556)            0.057269            0.171806   
303  (22352, 22356)     (20724)            0.066079            0.202643   
123         (21122)     (21121)            0.070485            0.127753   
458  (22629, 22326)     (22631)            0.061674            0.123348   
262  (22356, 20719)     (20724)            0.061674            0.202643   
340  (20726, 21559)     (22352)            0.061674            0.281938   
442  (22554, 21559)     (22352)            0.057269            0.281938   
404  (21080, 21086)     (21094)            0.083700            0.127753   
405  (21080, 21094)     (21086)            0.083700            0.140969   
244         (22555)     (22556)            0.083700            0.171806   

      support  confidence      lift  leverage  conviction  
471  0.057269    1.000000  5.820513  0.047430         inf  
303  0.066079    1.000000  4.934783  0.052689         